In [10]:
import h2o
import h2o.grid
import pandas as pd
import numpy as np

from h2o.estimators.glm import H2OGeneralizedLinearEstimator


In [6]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.11+9-LTS-194, mixed mode)
  Starting server from C:\Users\Gerd\venvs\h2o\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Gerd\AppData\Local\Temp\tmp1wwns7_e
  JVM stdout: C:\Users\Gerd\AppData\Local\Temp\tmp1wwns7_e\h2o_Gerd_started_from_python.out
  JVM stderr: C:\Users\Gerd\AppData\Local\Temp\tmp1wwns7_e\h2o_Gerd_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,2 months and 10 days
H2O_cluster_name:,H2O_from_python_Gerd_czc9mw
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.975 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"accepting new members, healthy"


In [7]:
data = h2o.import_file('http://h2o-public-test-data.s3.amazonaws.com/smalldata/airlines/allyears2k_headers.zip')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [8]:
train, valid, test = data.split_frame([0.8, 0.1], seed=69)

In [9]:
print(train.nrows, valid.nrows, test.nrows)

35255 4272 4451


In [12]:
target = "IsArrDelayed"
ignoreFields = [
    "ArrDelay", "DepDelay",
    "CarrierDelay", "WeatherDelay",
    "NASDelay", "SecurityDelay",
    "LateAircraftDelay", "IsDepDelay",
    "IsArrDelayed", "ActualElapsedTime"
]

xAll = [i for i in train.names if i not in ignoreFields]
xLikely = ["Month", "DayOfWeek", "UniqueCarrier",
          "Origin", "Dest", "Distance", "Cancelled", "Diverted"]

In [13]:
m_def = H2OGeneralizedLinearEstimator(family = "binomial",)

m_def.train(xAll, target, train, validation_frame = valid)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [14]:
m_def.logloss(valid=True)

0.5414570563465372

In [15]:
g = h2o.grid.H2OGridSearch(
    H2OGeneralizedLinearEstimator(
    family = "binomial",
    lambda_search = True
    ),
    hyper_params = {
        "alpha":[x * 0.01 for x in range (0, 100)],
    },
    search_criteria = {
        "strategy":"RandomDiscrete",
        "max_models":8,
        "max_runtime_secs":30
    }
)
g.train(xAll, target, train, validation_frame = valid)

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [16]:
g

                     alpha  \
0                   [0.85]   
1                   [0.84]   
2                   [0.73]   
3    [0.47000000000000003]   
4                   [0.26]   
5                    [0.2]   
6                   [0.15]   
7                   [0.02]   

                                                     model_ids  \
0  Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_4_model_6   
1  Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_4_model_2   
2  Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_4_model_1   
3  Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_4_model_7   
4  Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_4_model_8   
5  Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_4_model_4   
6  Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_4_model_3   
7  Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_4_model_5   

              logloss  
0  0.5050724912558561  
1  0.5051205759415903  
2  0.5057264406617276  
3   0.508035509352371  
4  0.511957279

In [17]:
g2 = h2o.grid.H2OGridSearch(
    H2OGeneralizedLinearEstimator(
    family = "binomial",
    lambda_search = True
    ),
    hyper_params = {
        "alpha":[x * 0.01 for x in range (0, 100)],
    },
    search_criteria = {
        "strategy":"Cartesian"
    }
)
g2.train(xAll, target, train, validation_frame = valid)

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [18]:
g2

        alpha                                                       model_ids  \
0       [0.0]    Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_21_model_1   
1      [0.99]  Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_21_model_100   
2      [0.98]   Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_21_model_99   
3      [0.97]   Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_21_model_98   
4      [0.96]   Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_21_model_97   
.. ..     ...                                                             ...   
95     [0.05]    Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_21_model_6   
96     [0.04]    Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_21_model_5   
97     [0.03]    Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_21_model_4   
98     [0.02]    Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_21_model_3   
99     [0.01]    Grid_GLM_py_2_sid_8dfd_model_python_1622892119670_21_model_2   

               logloss  
0 

In [19]:
h2o.cluster().shutdown()

H2O session _sid_8dfd closed.
